In [ ]:
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind


In [ ]:
# Load your last 100 games
with open("last_100_chess_games.json", "r") as file:
    games = json.load(file)


In [ ]:
# Extract relevant data
extracted_data = []
for game in games:
    color = "White" if game["white"]["username"].lower() == "samuraibartu" else "Black"
    result = game["white"]["result"] if color == "White" else game["black"]["result"]
    my_rating = game["white"]["rating"] if color == "White" else game["black"]["rating"]
    opponent_rating = game["black"]["rating"] if color == "White" else game["white"]["rating"]

    extracted_data.append({
        "Date": pd.to_datetime(game["end_time"], unit='s'),
        "Time Control": game.get("time_control"),
        "Time Class": game.get("time_class"),
        "Result": result,
        "My Color": color,
        "My Rating": my_rating,
        "Opponent Rating": opponent_rating,
        "Opponent Username": game["black"]["username"] if color == "White" else game["white"]["username"],
        "URL": game.get("url")
    })

df = pd.DataFrame(extracted_data)
df["Rating Diff"] = df["My Rating"] - df["Opponent Rating"]
df["Win"] = df["Result"].apply(lambda x: 1 if x == "win" else 0)

df.head()


In [ ]:
sns.set(style="whitegrid")
plt.figure(figsize=(6, 4))
sns.barplot(data=df, x="My Color", y="Win")
plt.title("Win Rate by Color")
plt.show()


In [ ]:
plt.figure(figsize=(6, 4))
sns.histplot(df["Rating Diff"], bins=20, kde=True)
plt.title("Rating Difference Distribution")
plt.show()


In [ ]:
plt.figure(figsize=(8, 4))
sns.barplot(data=df, x="Time Control", y="Win")
plt.title("Win Rate by Time Control")
plt.xticks(rotation=45)
plt.show()


In [ ]:
greater = df[df["Rating Diff"] > 0]["Win"]
less_equal = df[df["Rating Diff"] <= 0]["Win"]
t_stat, p_val = ttest_ind(greater, less_equal)

print("T-statistic:", round(t_stat, 3))
print("P-value:", round(p_val, 5))
if p_val < 0.05:
    print("✅ Rating difference has a statistically significant effect on win rate.")
else:
    print("❌ No significant effect detected.")
